In [3]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as sps
import tqdm
%matplotlib inline

In [205]:
def grad_descent(x0, alpha, precision, max_iters):
    cur_x = x0
    previous_step_size = 1
    iters = 0
    while previous_step_size > precision and iters < max_iters:
        prev_x = cur_x
        cur_x = cur_x + alpha * df(prev_x)
        previous_step_size = np.linalg.norm(cur_x - prev_x) 
        iters += 1
    
    print("The local maximum of grad occurs at", cur_x)
    return cur_x
    
def proj_grad_descent(x0, alpha, precision, max_iters):
    cur_x = x0
    previous_step_size = 1
    iters = 0
    while previous_step_size > precision and iters < max_iters:
        prev_x = cur_x
        cur_x = euclidean_proj_simplex(prev_x + alpha * df(prev_x))
        previous_step_size = np.linalg.norm(cur_x - prev_x)
        iters += 1
        
    print("The local maximum of proj grad occurs at", cur_x)
    return cur_x
    
def euclidean_proj_simplex(v, s=1):
    n, = v.shape  # will raise ValueError if v is not 1-D
    # check if we are already on the simplex
    if v.sum() == s and np.alltrue(v >= 0):
        # best projection: itself!
        return v
    # get the array of cumulative sums of a sorted (decreasing) copy of v
    u = np.sort(v)[::-1]
    cssv = np.cumsum(u)
    # get the number of > 0 components of the optimal solution
    rho = np.nonzero(u * np.arange(1, n+1) > (cssv - s))[0][-1]
    # compute the Lagrange multiplier associated to the simplex constraint
    theta = (cssv[rho] - s) / (rho + 1.0)
    # compute the projection by thresholding v using theta
    w = (v - theta).clip(min=0)
    return w

In [222]:
t_matrix = np.random.rand(5,5) 
t_matrix /= t_matrix.sum(axis=1)[:,None]

c_matrix = (t_matrix * np.log2(t_matrix)).sum(axis=-1)

f = lambda x: c_matrix.T @ x - ((t_matrix @ x) * np.log2(t_matrix @ x)).sum(axis=-1)

df = lambda x: c_matrix.T - (t_matrix * (np.log2(t_matrix @ x) + 1)).sum(axis=-1)

In [230]:
x0 = np.random.rand(5) * 5
x0 /= x0.sum()
print(x0)
alpha = 0.00001
precision = 10**-2
iters = 10**5

# x = grad_descent(x0, alpha, precision, iters)
# print("max grad capacity:", f(x))
x = proj_grad_descent(x0, alpha, precision, iters)
print("max proj grad capacity:", f(x))


[0.34469945 0.15227411 0.07701194 0.35510108 0.07091341]
The local maximum of proj grad occurs at [0.34470022 0.15227266 0.07701056 0.35510386 0.0709127 ]
max proj grad capacity: 0.32486706971757995


In [216]:
print(np.log2(-5))

nan


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in log2
  """Entry point for launching an IPython kernel.
